# Container Management
This notebook demonstrates how the Docker Python API can be used for container management for a Model Service API.

Requires `docker==3.7.2` which can be installed with `pip install docker==3.7.2`.

Also, this assumes that the containers you are interested in running are available as built images on the server where this code will be executed. If you haven't yet built the containers, you can with:

```
git clone --recursive https://gitlab.kimetrica.com/DARPA/darpa.git
cd darpa/kiluigi
git checkout master
cd ..

sed -i '4s/kiluigi\///g' env.example
cp env.example kiluigi/.env
cp luigi.cfg.example luigi.cfg
cd kiluigi
docker-compose run --entrypoint=bash scheduler
```

Once you have built the container and are inside it, exit the container, stop and remove all running Kiluigi related containers. You should then `cd ../..` so that you are one level above the `darpa` directory.

### Connect to Docker and ensure correct images exist

In [1]:
import docker
import re

In [2]:
client = docker.from_env()

Obtain available images:

In [3]:
kimetrica_images = ['drp_scheduler:latest', 'drp_db:latest']

In [4]:
images = [img.tags[0] for img in client.images.list()]

In [5]:
if set(kimetrica_images) - set(images) == set():
    print("Correct images are available.")
else:
    print(f"You need to build {kimetrica_images}")

Correct images are available.


### Set up environment variables

First we need a function to parse KiLuigi `.env` files:

In [6]:
def parse_env_file(path_to_file):
    '''
    Parse a Kimetrica .env file into a dictionary
    '''
    envre = re.compile(r'''^([^\s=]+)=(?:[\s"']*)(.+?)(?:[\s"']*)$''')
    env_var = re.compile(r'\$\{[0-9a-zA-Z_]*\}')
    alpha_num = re.compile(r'[0-9a-zA-Z_]+')
    result = {}
    with open(path_to_file) as ins:
        for line in ins:
            match = envre.match(line)
            if match is not None:
                key = match.group(1)
                val = match.group(2)
                to_replace = env_var.findall(val)
                for v in to_replace:
                    found = result[alpha_num.search(v).group(0)]
                    val = val.replace(v, found)
                result[key] = val
    return result

Now we can define a set of variables required for Docker. These include:

- Naming the scheduler and database
- Defining the scheduler entrypoint (the Luigi command to run; includes the specific model/task combination)
- Volumes to mount to the containers
- Reading in environment variables 
- Ports to expose on the database
- The name of our Docker network

In [7]:
scheduler = 'drp_scheduler:latest'
db = 'drp_db:latest'
entrypoint="luigi --module models.malnutrition_model.tasks models.malnutrition_model.tasks.RasterToCSV --local-scheduler"
volumes = {'/home/ubuntu/darpa/': {'bind': '/usr/src/app/', 'mode': 'rw'}}
environment = parse_env_file('darpa/kiluigi/.env')
db_ports = {'5432/tcp': 5432}
network_name = "kiluigi"

environment['PYTHONPATH'] = '/usr/src/app:/usr/src/app/kiluigi'

We need to set some specific environment variables for the db:

In [8]:
db_environment = {"APP": environment["APP"],
                  "ENV": environment["ENV"],
                  "PGPASSWORD": environment["PGPASSWORD"],
                  "POSTGRES_PASSWORD": environment["PGPASSWORD"]}

### Set up Docker Network
Create a network for Kiluigi activities. First, remove any lingering networks called `kiluigi`:

In [9]:
for net in client.networks.list():
    if net.name == network_name:
        client.networks.get(net.id).remove()

In [10]:
network = client.networks.create(network_name, driver="bridge")

### Run db
We now run the db and store its object as `db_container` in case we need it in the future:

In [11]:
containers = client.containers

In [12]:
db_container = containers.run(db, environment=db_environment, ports=db_ports, network=network_name, detach=True)

### Run Model

In [19]:
model = containers.run(scheduler, 
               environment=environment, 
               volumes=volumes, 
               network=network_name, 
               links={db_container.short_id: None},
               entrypoint=entrypoint,
               detach=True)

We run the model detached (as a daemon container) but can access its logs with:

In [27]:
model_logs = model.logs(stream=True)
for l in model_logs:
    print(l)

b'[2019-04-22 21:44:37,397] INFO [luigi:80] logging configured via *.conf file\n'
b'[2019-04-22 21:44:37,411] INFO [luigi-interface:579] Informed scheduler that task   models.malnutrition_model.tasks.RasterToCSV_0_5_travel_time___landcover____r_333a86b912   has status   PENDING\n'
b'[2019-04-22 21:44:37,909] INFO [luigi-interface:143] No prior data file exists at /tmp/data/CkanTarget/data.kimetrica.com/cached_ckan_metadata_kiluigi.pickle\n'
b'[2019-04-22 21:44:38,580] INFO [luigi-interface:128] Attempting to load from /tmp/data/CkanTarget/data.kimetrica.com/cached_ckan_metadata_kiluigi.pickle\n'
b'[2019-04-22 21:44:39,059] INFO [luigi-interface:128] Attempting to load from /tmp/data/CkanTarget/data.kimetrica.com/cached_ckan_metadata_kiluigi.pickle\n'
b'[2019-04-22 21:44:39,586] INFO [luigi-interface:128] Attempting to load from /tmp/data/CkanTarget/data.kimetrica.com/cached_ckan_metadata_kiluigi.pickle\n'
b'[2019-04-22 21:44:40,088] INFO [luigi-interface:128] Attempting to load from /t

### Container status

We can check the container's status by reloading its representation from Docker then asking for its `status`:

In [43]:
model.reload()

In [44]:
model.status

'exited'

We can compare this with the `status` of the database, which should continue running until we stop it:

In [46]:
db_container.reload()
db_container.status

'running'

We can shut off the `db_container` at this point:

In [47]:
db_container.stop()

We can recheck its status:

In [48]:
db_container.reload()
db_container.status

'exited'

### Pruning stopped containers
Finally, we should prune (remove) containers that have been stopped:

In [49]:
containers.prune()

{'ContainersDeleted': ['a06f7baa207f92a7bc341092bb5dc874d00710fbedbdb39a2b1f1cc00c19d396',
  'dfb790fe6421b10ef79adbb83e9f577be9429ee0ffd3540c5e35ad612f80685b',
  'b38c063518e0e9e89b877fe02b88a1cc95a7e9b30c098c35e69d2a38b9988c70',
  '51467d4160fb53e736b587bdddef61d24af8dfa9fd819b5083481eea8deb5596'],
 'SpaceReclaimed': 6444046321}